In [36]:
import urllib.request
import pandas as pd

In [37]:
id_uni = 190

In [38]:
db_players = f'https://s{id_uni}-fr.ogame.gameforge.com/api/players.xml'  # une fois par jour
db_ladder = f'https://s{id_uni}-fr.ogame.gameforge.com/api/highscore.xml?category=1&type=1' # par heure
# category : 1 = Player, 2 = Alliance
type = {0 : 'Total',
        1 : 'Eco',
        2 : 'Research',
        3 : 'Militaire',
        4 : 'Militaire lost',
        5 : 'Militaire construit',
        6 : 'Militaire Détruit',
        7 : 'Honor',
        8 : 'FdV',
        9 : 'Eco FdV',
        10 : 'Techno FdV',
        11 : 'Exploration FdV'}

# On récupère la data

In [39]:
urllib.request.urlretrieve(db_players, "xml/players.xml")

('xml/players.xml', <http.client.HTTPMessage at 0x13178e07fa0>)

In [40]:
for key, value in type.items():
    db = f'https://s{id_uni}-fr.ogame.gameforge.com/api/highscore.xml?category=1&type={key}'
    urllib.request.urlretrieve(db, f"xml/ladder_{value}.xml")

# Chargement

In [41]:
df_players = pd.read_xml('xml/players.xml')

# Modif

In [42]:
# Osef de l'alliance
df_players.drop('alliance', axis=1, inplace=True)

In [43]:
mapping = {'a' : 'admin',
           'v' : 'mv',
           'i' : 'inactif',
           None: 'actif'}
df_players['status'] = df_players['status'].replace(mapping)
df_players

,id,name,status
0,1,Legor,admin
1,100000,GameAdmin,admin
2,100003,comagf,admin
3,100004,OGameTeam,admin
4,100006,SeumMan,mv
...,...,...,...
4293,105683,Constable Aries,actif
4294,105684,Czar Midas,actif
4295,105685,Ganinox,actif
4296,105686,Renegade Infrared,actif


# On fusionne

In [44]:
for value in type.values():
    df = pd.read_xml(f'xml/ladder_{value}.xml')
    if len(df.columns) == 3:
        df.columns = [f'position_{value}', 'id', f'points_{value}']
    else:
        df.columns = [f'position_{value}', 'id', f'points_{value}', "ships"]
    df_players = pd.merge(df_players, df, on='id')

In [45]:
df_players

,id,name,status,position_Total,points_Total,position_Eco,points_Eco,position_Research,points_Research,position_Militaire,...,position_Honor,points_Honor,position_FdV,points_FdV,position_Eco FdV,points_Eco FdV,position_Techno FdV,points_Techno FdV,position_Exploration FdV,points_Exploration FdV
0,100006,SeumMan,mv,33,1965914,82,560201,42,158330,15,...,4304,-49544,48,105980,48,79618,50,26357,342,5
1,100007,Le Vrai Australien,actif,35,1828711,26,989013,23,215346,46,...,4287,-9733,121,39810,139,20007,74,19794,223,9
2,100008,La VenDeTTa,mv,478,75304,719,18208,563,7788,368,...,549,24,831,549,805,496,722,50,507,2
3,100009,dylou,inactif,711,31060,643,22619,655,5850,2114,...,231,722,474,2589,417,2589,904,0,692,0
4,100010,Belle Alice,mv,1206,6094,1083,5624,1471,454,1711,...,904,1,2284,0,2283,0,905,0,693,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,105683,Constable Aries,actif,2862,2,2876,2,4274,0,4274,...,3992,0,2414,0,2413,0,4273,0,4273,0
4273,105684,Czar Midas,actif,3034,1,3028,1,4275,0,4275,...,3993,0,4294,0,4294,0,4274,0,4274,0
4274,105685,Ganinox,actif,2642,4,2633,4,4276,0,4276,...,3994,0,2862,0,2861,0,4275,0,4275,0
4275,105686,Renegade Infrared,actif,4300,0,4299,0,4277,0,4277,...,3995,0,4295,0,4295,0,4276,0,4276,0


# On cherche les meilleurs ina

In [51]:
pd.set_option('display.max_rows', None)
df_ina = df_players[df_players['status'] == 'inactif']
df_ina.sort_values('points_Total', inplace=True, ascending=False)
df_ina.to_excel('inactif/ina.xlsx', index=False)
df_ina.head(30)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_5544\191281827.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ina.sort_values('points_Total', inplace=True, ascending=False)


,id,name,status,position_Total,points_Total,position_Eco,points_Eco,position_Research,points_Research,position_Militaire,...,position_Honor,points_Honor,position_FdV,points_FdV,position_Eco FdV,points_Eco FdV,position_Techno FdV,points_Techno FdV,position_Exploration FdV,points_Exploration FdV
133,100170,Capell,inactif,462,78711,383,68152,549,8124,734,...,148,1835,486,2434,425,2434,967,0,765,0
445,100588,Chasseur Nora,inactif,517,63473,450,49657,426,12301,1337,...,20,11914,571,1514,524,1454,711,60,956,0
304,100398,TH3 Kill3R,inactif,696,32563,713,18935,408,13261,2202,...,285,334,924,367,942,290,698,69,277,7
3,100009,dylou,inactif,711,31060,643,22619,655,5850,2114,...,231,722,474,2589,417,2589,904,0,692,0
57,100074,KidPatt,inactif,770,25213,662,21492,801,3717,1139,...,310,236,1778,3,1778,3,926,0,720,0
23,100031,Engineer Pluto,inactif,812,22453,733,17765,759,4280,1226,...,597,18,897,407,918,316,676,91,703,0
1317,101860,Roy,inactif,935,15569,964,8454,593,7115,1946,...,199,1182,2380,0,2379,0,1692,0,1568,0
471,100624,Moretos,inactif,997,12809,1069,5933,698,5054,2074,...,395,86,546,1821,514,1528,548,282,200,11
685,100923,Yoloooo,inactif,1030,11315,1090,5525,846,3250,2328,...,246,592,479,2539,597,1004,319,1534,1112,0
1456,102111,Sovereign Ganimed,inactif,1109,8419,1004,7155,1207,1087,1373,...,1748,0,1133,176,1128,164,861,8,427,4
